In [1]:
import jarvis
import pandas as pd
import numpy as np

# Where do I start? Summarize current and past experiments

In [2]:
summary = jarvis.versionSummaries('twitter_demo')

100%|██████████| 2/2 [00:11<00:00,  5.67s/it]


In [3]:
summary

,__trialNum__,alpha,frac,model,model_accuracy,split_seed,__commitHash__
0,0,0.000000,0.75,. . .,0.78794,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
1,7,1.000000,0.75,. . .,0.66765,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
2,6,0.857143,0.75,. . .,0.67680,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
3,1,0.142857,0.75,. . .,0.75478,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
4,4,0.571429,0.75,. . .,0.69731,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
5,3,0.428571,0.75,. . .,0.71146,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
6,2,0.285714,0.75,. . .,0.73327,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
7,5,0.714286,0.75,. . .,0.68776,42,4646a6c2aca1d86aa7b9a62136b0cf592268f201
8,0,0.000000,0.75,. . .,0.79514,42,41609fa5f64f2e5befdbdff644cd9d3bf913f8f4
9,7,1.000000,0.75,. . .,0.57407,42,41609fa5f64f2e5befdbdff644cd9d3bf913f8f4


# What is the best experiment?
Here, we measure _goodness_ of an experiment by its max model accuracy, over all trials.

In [4]:
summary_stat = summary.groupby('__commitHash__')['model_accuracy'].max()
summary_stat

__commitHash__
41609fa5f64f2e5befdbdff644cd9d3bf913f8f4    0.79514
4646a6c2aca1d86aa7b9a62136b0cf592268f201    0.78794
Name: model_accuracy, dtype: float64

In [5]:
best_commit_hash = summary_stat.idxmax()
best_commit_hash[0:5] + '...'

'41609...'

# What is the best trial, given the best experiment?

In [6]:
best_index = summary.loc[summary['__commitHash__'] == best_commit_hash]['model_accuracy'].idxmax()
best_trial = summary.iloc[best_index]['__trialNum__']
best_trial

'0'

# So the former experiment is better than the latter. What changed?

In [7]:
versions = summary.__commitHash__.unique()
most_recent, least_recent = versions

In [8]:
jarvis.diffExperimentVersions('twitter_demo', least_recent, most_recent)

train_model.py --> train_model.py
@@ -47,8 +47,8 @@ def train(tweet_df, alpha):
     ## Convert tweet to bag of words for learning
 
     # Tokenize Text
-    vectorizer = TfidfVectorizer()
-    #vectorizer = CountVectorizer()
+    #vectorizer = TfidfVectorizer()
+    vectorizer = CountVectorizer()
 
     X_train = vectorizer.fit_transform(tweet_df["tweet"])
 


# Let's get the best model yet!
From the best trial, from the best experiment

In [9]:
best_model = jarvis.checkoutArtifact('twitter_demo', best_trial, best_commit_hash, 'model.pkl')

# Here's the best model in action

In [10]:
country_dict = best_model['country_dict']
classifier = best_model['classifier']
vectorizer = best_model['vectorizer']

code_dict = {}

for kee in country_dict:
    code_dict[country_dict[kee]] = kee

while True:
    tweet = input("What's on your mind? ")
    if tweet == 'exit':
        break
    tweet_vec = vectorizer.transform(np.array([tweet,]))
    country_id = classifier.predict(tweet_vec)
    print("Predicted country of origin: {}\n".format(code_dict[country_id[0]]))

What's on your mind? Hello world
Predicted country of origin:  US

What's on your mind? tqm
Predicted country of origin:  MX

What's on your mind? exit
